# Tutoriel 1 - Modèle Bag of Words (BOW) à l'aide d'NLTK et pandas

### 0. Téléchargement d'un livre éléctronique

In [ ]:
from urllib import request
url = "http://www.gutenberg.org/files/2554/2554-0.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')
print(raw[:100])

### 1. Tokenization de phrases et création d'un DataFrame Pandas

On utilise le Sentence Tokenizer d'NLTK pour séparer le text brut en phrases. Chaque phrase ainsi obtenue correspond à une rangée du DataFrame.

df.head() permet d'afficher les premières rangées (5 par défaut) d'un DataFrame.

In [ ]:
import pandas as pd
from nltk.tokenize import sent_tokenize
import nltk

nltk.download("punkt")

df = pd.DataFrame({"documents": sent_tokenize(raw)})
df.head(10)

### 2. Tokenization de mots et fonction apply

Les documents (phrases) sont séparés en token à l'aide de l'outil word_tokenize. Cette opération est vectorisée et appliquée à l'ensemble des lignes du DataFrame avec l'opération "apply".

À noter que dans ce contexte, la valeur assignée à la colonne "words" d'une rangée correspond à celle retournée par la fonction word_tokenize si on lui passait la valeur "documents" de la même rangée.

In [ ]:
from nltk.tokenize import word_tokenize

df["words"] = df["documents"].apply(word_tokenize)
print(df["words"][0])
print(type(df["words"][0]))
df.head()

### 3. Retrait des stopwords

Une fonction lambda est appliquée sur la colonne "words" de chaque rangée avec apply. La fonction lambda permet de manipuler explicitement la valeur existante d'une rangée.

La manipulation effectuée dans cet exemple est la création d'une liste de mots sans stopwords, en reprenant chaque élément de la colonne "words" et ne gardant ceux ne figurant pas dans la liste des stopwords d'NLTK.



In [ ]:

from nltk.corpus import stopwords
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))

df["stopwords_removed"] = df["words"].apply(lambda words: [word for word in words if word not in stop_words])
df.head()

### 4. Stemming et Lemmatization

Utilisation des librairies de NLTK pour stemmer ou lemmatizer les listes de mots de chaque phrase.

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

df["porter_stemmed"] = df["stopwords_removed"].apply(lambda words: [ps.stem(word) for word in words])

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download("wordnet")
lem = WordNetLemmatizer()

df["word_net_lemm"] = df["stopwords_removed"].apply(lambda words: [lem.lemmatize(word) for word in words])
df.head()

### 5. Modèle Bag of Words

Calcul de la fréquence des tokens dans le corpus, dont on garde les N (200 dans l'exemple) token les plus fréquents.

Les phrases sont représentées par un vecteur de taille N: à chaque indice, 1 indique que ce Nième mot le plus fréquent est dans ladite phrase, 0 indique qu'il est absent de la phrase.

Notez l'utilisation de la fonction "value_counts" sur une série Pandas; celle-ci retourne un décompte des valeurs les plus fréquentes d'une colonne.

In [ ]:
# décompte de tous les tokens dans le corpus entier
vocab_words = pd.Series(word_tokenize(raw))
vocab_words = [word for word in vocab_words if word not in stop_words]
vocab_words = [lem.lemmatize(word) for word in vocab_words]

# 200 mots les plus fréquents
token_frequencies = pd.Series(vocab_words).value_counts().head(200).to_dict()

In [ ]:
# 1 si le token parmi les plus fréquents est dans la phrase, 0 sinon
def sentence_vector(sentence, token_frequencies):
  return [1 if token in sentence else 0 for token in token_frequencies.keys()]

In [ ]:
df["BOWrepresentation"] = df.word_net_lemm.apply(lambda x: sentence_vector(x, token_frequencies))
df.head()

#### Représentation des BOW en matrice numpy

In [ ]:
import numpy as np

np.asarray(list(df.BOWrepresentation.values))

#### 6. Modèle TF-IDF avec sklearn


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

df["joined_lemmas"] = df.word_net_lemm.apply(lambda x: " ".join(x))

tfidf = vectorizer.fit_transform(df["joined_lemmas"].values)
feature_names = vectorizer.get_feature_names()
tfidf = tfidf.toarray()
tfidf

### 7. Distance entre deux représentations de documents

La librairie scipy offre des outils pour les calculs de distance entre vecteurs unidimensionnels. Par exemple, voici comment calculer la distance cosinus entre les représentations TF-IDF des deux premières phrases du livre:

In [ ]:
from scipy.spatial.distance import cosine

cosine(tfidf[0], tfidf[1])